# Waffle Chart in Plotly

After learning how to do waffle charts in Tableau, I tried the same using [Plotly](https://plotly.com/python/). Any example I found on the web did not generate what I had in mind, so here is my take.

The waffle plot is kind of a squared alternative to pie charts. It is composed of 100 little squares in a 10×10 grid.

Example data: Berlin results of the European elections 2024.

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
df = pd.read_csv("data/berlin.csv", skiprows=4, skipfooter=2)
df.tail()

/tmp/ipykernel_22173/3441966150.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("data/berlin.csv", skiprows=4, skipfooter=2)


,Partei,Anzahl,Anteil,Gewinn und Verlust in %-Punkten
29,KLIMALISTE,2653,"0,2 %","0,2"
30,LETZTE GENERATION,10005,"0,6 %","0,6"
31,PDV,1056,"0,1 %","0,1"
32,PdF,8491,"0,5 %","0,5"
33,V-Partei,2198,"0,1 %","0,1"


Replace party name in rows with less than 1 % with "Other"

In [3]:
total = df["Anzahl"].sum()



In [4]:
df.loc[df["Anzahl"] < (total / 100), "Partei"] = "Other"

In [5]:
categories = df["Partei"].unique()

# Make sure "Other" is last
categories = np.append(categories[categories != "Other"], "Other")


For the tooltip, we need the max string length (and add some characters for the percentage). 

In [6]:
M = max([len(s) for s in categories]) + 10

Two numpy arrays with one entry for each little square, `waffledata` will contain a integer representing each party, `customdata` will contain the tooltip string.

In [7]:
waffledata = np.zeros(100)
customdata = np.empty(100, dtype=f'<U{M}')

Fill the arrays with data

In [8]:
cum = 0

for i, cat in enumerate(categories):
    qty = df[df["Partei"] == cat]["Anzahl"].sum()
    percentage = 100 * qty / total
    start = round(cum)
    stop = round(cum + percentage)
    if stop > 100:
        stop = 100

    waffledata[start:stop] = i
    customdata[start:stop] = f"{cat}<br>{percentage:.1f} %"
    cum += percentage


Reshape

In [9]:
waffledata = waffledata.reshape(10, 10)
customdata = customdata.reshape(10, 10)

In [10]:
colorscale = px.colors.qualitative.Dark2
gap = 2


fig = go.Figure(
    go.Heatmap(
        z=waffledata,
        xgap=gap,
        ygap=gap,
        colorscale=colorscale,
        customdata=customdata,
        hovertemplate="%{customdata}<extra></extra>",
        showscale=False,
    )
)

Let's have a look at what we have until now.

In [11]:
fig.show()

It should not have axes labels and it should be square.

In [12]:
fig.update_layout(
        # Waffle without axes and must be square
        yaxis=dict(visible=False, constrain='domain', scaleanchor="x"),
        xaxis=dict(visible=False, constrain='domain',),

        title="Berlin, European Election Results 2024",
        title_x=0.5, # Center the title

        width=500, 
        height=500, 
        # yaxis_autorange='reversed' # Optional: reverse the waffle

    )


fig.show()

The colors are a bit random. You can provide a list with css named colors or color hex codes. 

In [13]:
categories

array(['GRÜNE', 'CDU', 'SPD', 'DIE LINKE', 'AfD', 'Die PARTEI', 'FDP',
       'Tierschutzpartei', 'MERA25', 'Volt', 'BSW', 'Other'], dtype=object)

In [14]:
colorscale = [
    "#66b266",  
    "#666666",  
    "#cc6666", 
    "#996699",  
    "#6666cc",  
    "#cc9966", 
    "#cccc66",  
    "#669999",  
    "#cc6666",  
    "#996699",  
    "#cc99cc",  
    "#999999"   
]

In [15]:
fig.update_traces(colorscale=colorscale)
fig.show()

In [17]:
# fig.write_html("waffle.html")